In [58]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [59]:
import pandas as pd
import os
import shutil
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split

In [60]:
df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-15_galaxy_zoo_ukidss_classifications.csv')
image_root = '/home/walml/repos/galaxy-datasets/derived_data/ukidss'




# nope, there rest are not here - though possibly they are simply on a different S3 subfolder. I can check with Zach.



# df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-15_galaxy_zoo_illustris_classifications.csv')
# image_root = '/home/walml/repos/galaxy-datasets/derived_data/illustris'

# df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-15_galaxy_zoo_gama09_classifications.csv')
# image_root = '/home/walml/repos/galaxy-datasets/derived_data/gama09'

# df = pd.read_csv('/media/walml/mirror/galaxy_zoo/older_projects/weekly_dumps/2017-10-22_galaxy_zoo_gama12_classifications.csv')
# image_root = '/home/walml/repos/galaxy-datasets/derived_data/gama12'

In [61]:
df.sample()

,id,subject_id,user,created_at,lang,user_agent,ukidss-0,ukidss-1,ukidss-2,ukidss-3,ukidss-4,ukidss-5,ukidss-6,ukidss-7,ukidss-8,ukidss-9,ukidss-10,ukidss-11
1812150,5314647e41ecea3012000175,524482bb3ae74054bf0038d8,vatanen82,2014-03-03 11:16:14 UTC,en,unknown,a-0,NaN,NaN,NaN,NaN,a-0,a-0;x-5,a-0,NaN,NaN,NaN,a-1


In [62]:
subject_df = df[['subject_id']].drop_duplicates()
len(subject_df)

70987

In [63]:
subject_df

,subject_id
0,524482ba3ae74054bf00031c
1,524482ba3ae74054bf0016bc
2,524482ba3ae74054bf002036
3,524482bc3ae74054bf00ae78
4,524482bc3ae74054bf00e900
...,...
1809155,524482bc3ae74054bf00ad84
1840102,524482bb3ae74054bf00518d
1869150,524482bc3ae74054bf00e3cd
1931337,524482bd3ae74054bf010e0c


In [64]:

subject_df['subfolder'] = subject_df['subject_id'].apply(lambda x: x[:21])
# subject_df['file_exists']
# subject_df['subfolder'].value_counts()

In [65]:
subject_df['url'] = 'https://s3.amazonaws.com/www.galaxyzoo.org/subjects/standard/' + subject_df['subject_id'] + '.jpg'
subject_df['url'].iloc[0]

'https://s3.amazonaws.com/www.galaxyzoo.org/subjects/standard/524482ba3ae74054bf00031c.jpg'

In [66]:
# subject_df['file_loc'] = image_root + '/' + subject_df['subfolder'] + '/' + subject_df['subject_id'] + '.jpg'
# subject_df['file_loc'].iloc[0]

In [67]:
# pull all of these with wget
with open(image_root + '/latest_image_urls.txt', 'w') as f:
    # missing files only
    # f.write('\n'.join(df[~df['file_exists']]['url'].values))
    f.write('\n'.join(subject_df['url'].values))

In [68]:
subject_df['filename'] = subject_df['subject_id'] + '.jpg'

In [69]:
# cat latest_image_urls.txt  | xargs -n10 -P4 wget --continue --quiet

In [70]:
reduced_votes_loc = '/home/walml/repos/gz-panoptes-reduction/data/ouroborous/ukidss/reduced_votes.csv'
votes = pd.read_csv(reduced_votes_loc)

In [71]:
votes

,subject_id,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,how-rounded_round,how-rounded_in-between,how-rounded_cigar,disk-edge-on_yes,disk-edge-on_no,bulge-shape_round,...,spiral-arm-count_more-than-4_fraction,spiral-arm-count_cant-tell_fraction,bulge-size_total-votes,bulge-size_no_fraction,bulge-size_just-noticeable_fraction,bulge-size_obvious_fraction,bulge-size_dominant_fraction,something-odd_total-votes,something-odd_yes_fraction,something-odd_no_fraction
0,524482ba3ae74054bf000001,15,24,1,0,0,15,23,1,6,...,0.0,0.000000,1,0.000000,1.000000,0.000000,0.000000,39,0.076923,0.923077
1,524482ba3ae74054bf000002,9,28,3,0,2,7,21,7,17,...,0.0,0.000000,7,0.285714,0.142857,0.428571,0.142857,37,0.243243,0.756757
2,524482ba3ae74054bf000003,30,1,9,28,2,0,0,1,0,...,0.0,0.000000,1,0.000000,0.000000,1.000000,0.000000,31,0.000000,1.000000
3,524482ba3ae74054bf000004,28,4,8,22,6,0,1,3,1,...,0.0,0.000000,3,0.000000,0.000000,0.666667,0.333333,32,0.062500,0.937500
4,524482ba3ae74054bf000005,35,4,1,16,19,0,0,4,0,...,0.0,0.000000,4,0.000000,0.000000,0.750000,0.250000,39,0.051282,0.948718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70982,524482bd3ae74054bf011588,30,4,6,3,26,1,2,2,0,...,0.0,0.000000,2,0.500000,0.500000,0.000000,0.000000,34,0.058824,0.941176
70983,524482bd3ae74054bf011589,32,5,3,26,6,0,0,5,0,...,0.0,1.000000,5,0.000000,0.400000,0.600000,0.000000,37,0.081081,0.918919
70984,524482bd3ae74054bf01158a,33,1,6,10,23,0,0,1,0,...,0.0,0.000000,1,0.000000,0.000000,1.000000,0.000000,34,0.029412,0.970588
70985,524482bd3ae74054bf01158b,29,9,2,0,27,2,3,6,3,...,0.0,0.500000,6,0.000000,0.666667,0.333333,0.000000,38,0.078947,0.921053


In [72]:
df = pd.merge(subject_df, votes)
df

,subject_id,subfolder,url,filename,smooth-or-featured_smooth,smooth-or-featured_featured-or-disk,smooth-or-featured_artifact,how-rounded_round,how-rounded_in-between,how-rounded_cigar,...,spiral-arm-count_more-than-4_fraction,spiral-arm-count_cant-tell_fraction,bulge-size_total-votes,bulge-size_no_fraction,bulge-size_just-noticeable_fraction,bulge-size_obvious_fraction,bulge-size_dominant_fraction,something-odd_total-votes,something-odd_yes_fraction,something-odd_no_fraction
0,524482ba3ae74054bf00031c,524482ba3ae74054bf000,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482ba3ae74054bf00031c.jpg,37,1,2,25,12,0,...,0.0,0.0,1,0.000000,0.000000,1.000,0.000,38,0.026316,0.973684
1,524482ba3ae74054bf0016bc,524482ba3ae74054bf001,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482ba3ae74054bf0016bc.jpg,27,5,8,26,1,0,...,0.0,0.0,4,0.000000,0.500000,0.500,0.000,32,0.062500,0.937500
2,524482ba3ae74054bf002036,524482ba3ae74054bf002,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482ba3ae74054bf002036.jpg,21,18,1,0,11,10,...,0.0,0.0,1,0.000000,0.000000,1.000,0.000,39,0.102564,0.897436
3,524482bc3ae74054bf00ae78,524482bc3ae74054bf00a,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482bc3ae74054bf00ae78.jpg,35,4,2,32,3,0,...,1.0,0.0,4,0.000000,0.000000,0.250,0.750,39,0.076923,0.923077
4,524482bc3ae74054bf00e900,524482bc3ae74054bf00e,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482bc3ae74054bf00e900.jpg,25,11,4,2,22,1,...,0.0,0.5,8,0.000000,0.000000,0.125,0.875,36,0.194444,0.805556
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70982,524482bc3ae74054bf00ad84,524482bc3ae74054bf00a,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482bc3ae74054bf00ad84.jpg,35,2,3,19,16,0,...,0.0,0.0,2,0.000000,0.000000,1.000,0.000,37,0.081081,0.918919
70983,524482bb3ae74054bf00518d,524482bb3ae74054bf005,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482bb3ae74054bf00518d.jpg,35,2,3,23,12,0,...,0.0,0.0,2,0.000000,0.000000,1.000,0.000,37,0.081081,0.918919
70984,524482bc3ae74054bf00e3cd,524482bc3ae74054bf00e,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482bc3ae74054bf00e3cd.jpg,20,10,10,2,17,1,...,0.0,0.0,9,0.333333,0.666667,0.000,0.000,30,0.933333,0.066667
70985,524482bd3ae74054bf010e0c,524482bd3ae74054bf010,https://s3.amazonaws.com/www.galaxyzoo.org/sub...,524482bd3ae74054bf010e0c.jpg,32,1,8,29,3,0,...,0.0,0.0,1,0.000000,1.000000,0.000,0.000,33,0.030303,0.969697


In [73]:
df['raw_file_loc'] = '/home/walml/repos/galaxy-datasets/derived_data/ukidss/' + df['subject_id'] + '.jpg'
df['file_loc'] = '/home/walml/repos/galaxy-datasets/derived_data/ukidss_final/' + df['subfolder'] + '/' + df['subject_id'] + '.jpg'


In [74]:
df['raw_file_loc'].iloc[0]

'/home/walml/repos/galaxy-datasets/derived_data/ukidss/524482ba3ae74054bf00031c.jpg'

In [75]:
df['file_exists'] = df['raw_file_loc'].apply(os.path.isfile)
df['file_exists'].mean()

1.0

In [76]:
# t = df[:1000]
# 60k images to check so takes a moment
df['image_sized_as_expected'] = df['file_loc'].apply(lambda x: Image.open(x).size == (424, 424))
df['image_sized_as_expected'].mean()

0.9979996337357544

In [77]:
df = df.query('image_sized_as_expected')

In [78]:
# drop some images which are not 424x424


In [79]:
from galaxy_datasets.shared import label_metadata

label_cols = label_metadata.ukidss_ortho_label_cols

# label_metadata.gz_ukidss_ortho_dependencies

In [80]:
label_cols

['smooth-or-featured-ukidss_smooth',
 'smooth-or-featured-ukidss_featured-or-disk',
 'smooth-or-featured-ukidss_artifact',
 'disk-edge-on-ukidss_yes',
 'disk-edge-on-ukidss_no',
 'has-spiral-arms-ukidss_yes',
 'has-spiral-arms-ukidss_no',
 'bar-ukidss_yes',
 'bar-ukidss_no',
 'bulge-size-ukidss_dominant',
 'bulge-size-ukidss_obvious',
 'bulge-size-ukidss_just-noticeable',
 'bulge-size-ukidss_no',
 'something-odd-ukidss_yes',
 'something-odd-ukidss_no',
 'how-rounded-ukidss_round',
 'how-rounded-ukidss_in-between',
 'how-rounded-ukidss_cigar',
 'bulge-shape-ukidss_round',
 'bulge-shape-ukidss_boxy',
 'bulge-shape-ukidss_no-bulge',
 'spiral-winding-ukidss_tight',
 'spiral-winding-ukidss_medium',
 'spiral-winding-ukidss_loose',
 'spiral-arm-count-ukidss_1',
 'spiral-arm-count-ukidss_2',
 'spiral-arm-count-ukidss_3',
 'spiral-arm-count-ukidss_4',
 'spiral-arm-count-ukidss_more-than-4',
 'spiral-arm-count-ukidss_cant-tell']

In [81]:
df = df.rename(columns=dict(zip([col.replace('-ukidss', '') for col in label_cols], label_cols)))
df = df.rename(columns=dict(zip([col.replace('-ukidss', '') + '_fraction' for col in label_cols], [col + '_fraction' for col in label_cols])))

for col in df.columns.values:
    if 'total-votes' in col:
        del df[col]

In [82]:
df.columns.values

array(['subject_id', 'subfolder', 'url', 'filename',
       'smooth-or-featured-ukidss_smooth',
       'smooth-or-featured-ukidss_featured-or-disk',
       'smooth-or-featured-ukidss_artifact', 'how-rounded-ukidss_round',
       'how-rounded-ukidss_in-between', 'how-rounded-ukidss_cigar',
       'disk-edge-on-ukidss_yes', 'disk-edge-on-ukidss_no',
       'bulge-shape-ukidss_round', 'bulge-shape-ukidss_boxy',
       'bulge-shape-ukidss_no-bulge', 'bar-ukidss_yes', 'bar-ukidss_no',
       'has-spiral-arms-ukidss_yes', 'has-spiral-arms-ukidss_no',
       'spiral-winding-ukidss_tight', 'spiral-winding-ukidss_medium',
       'spiral-winding-ukidss_loose', 'spiral-arm-count-ukidss_1',
       'spiral-arm-count-ukidss_2', 'spiral-arm-count-ukidss_3',
       'spiral-arm-count-ukidss_4', 'spiral-arm-count-ukidss_more-than-4',
       'spiral-arm-count-ukidss_cant-tell', 'bulge-size-ukidss_no',
       'bulge-size-ukidss_just-noticeable', 'bulge-size-ukidss_obvious',
       'bulge-size-ukidss_domin

In [83]:
df[label_cols]

,smooth-or-featured-ukidss_smooth,smooth-or-featured-ukidss_featured-or-disk,smooth-or-featured-ukidss_artifact,disk-edge-on-ukidss_yes,disk-edge-on-ukidss_no,has-spiral-arms-ukidss_yes,has-spiral-arms-ukidss_no,bar-ukidss_yes,bar-ukidss_no,bulge-size-ukidss_dominant,...,bulge-shape-ukidss_no-bulge,spiral-winding-ukidss_tight,spiral-winding-ukidss_medium,spiral-winding-ukidss_loose,spiral-arm-count-ukidss_1,spiral-arm-count-ukidss_2,spiral-arm-count-ukidss_3,spiral-arm-count-ukidss_4,spiral-arm-count-ukidss_more-than-4,spiral-arm-count-ukidss_cant-tell
0,37,1,2,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,27,5,8,1,4,0,4,0,4,0,...,0,0,0,0,0,0,0,0,0,0
2,21,18,1,17,1,1,0,1,0,0,...,2,0,1,0,0,1,0,0,0,0
3,35,4,2,0,4,1,3,0,4,3,...,0,1,0,0,0,0,0,0,1,0
4,25,11,4,3,8,4,4,1,7,7,...,0,1,1,2,0,2,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70982,35,2,3,0,2,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0
70983,35,2,3,0,2,0,2,0,2,0,...,0,0,0,0,0,0,0,0,0,0
70984,20,10,10,1,9,2,7,0,9,0,...,0,0,0,2,2,0,0,0,0,0
70985,32,1,8,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# base_path = '/home/walml/repos/galaxy-datasets/derived_data/ukidss_final/'

# for _, galaxy in df.iterrows():
#     if not os.path.isdir(base_path + galaxy['subfolder']):
#         os.mkdir(base_path + galaxy['subfolder'])
#     shutil.copyfile(galaxy['raw_file_loc'], galaxy['file_loc'])

# # warning - the pictures seem to be VERY smooth. not sure how useful these will be

In [85]:
train_catalog, test_catalog = train_test_split(df, test_size=0.2, random_state=42)

In [86]:
train_catalog.to_parquet('/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_ukidss/ukidss_train_catalog.parquet', index=False)
test_catalog.to_parquet('/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_ukidss/ukidss_test_catalog.parquet', index=False)

/home/walml/miniforge3/envs/rings/lib/python3.9/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [87]:
import hashlib

for loc in [
    '/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_ukidss/ukidss_train_catalog.parquet',
    '/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_ukidss/ukidss_test_catalog.parquet',
    '/home/walml/Dropbox (The University of Manchester)/pytorch-galaxy-datasets/gz_ukidss/images.tar.xz'  # compressed from images above
]:
# for loc in [
#     '/home/walml/repos/galaxy-datasets/roots/gz_ukidss/ukidss_train_catalog.parquet',
#     '/home/walml/repos/galaxy-datasets/roots/gz_ukidss/ukidss_test_catalog.parquet',
#     '/home/walml/repos/galaxy-datasets/roots/gz_ukidss/images.tar.xz'  # compressed from images above
# ]:
# for loc in [
#     '/home/walml/repos/galaxy-datasets/roots/gz_ukidss_debug/ukidss_train_catalog.parquet',
#     # '/home/walml/repos/galaxy-datasets/roots/gz_ukidss_debug/ukidss_test_catalog.parquet',
#     # '/home/walml/repos/galaxy-datasets/roots/gz_ukidss_debug/images.tar.xz'  # compressed from images above
#     '/home/walml/repos/galaxy-datasets/derived_data/ukidss_train_catalog.parquet'
# ]:
    # print hash
    with open(loc, 'rb') as f:
        md5_checksum = hashlib.md5(f.read()).hexdigest()

    print(md5_checksum)

8d84de1a1f33556f83470ce0c22c979b
f64b850e941b09546df5aca84b708fb4


d4586af013b1543753346544b4703711
